# Multi-Phase Flow Example

This notebook demonstrates various multi-phase flow phenomena in PHASTA, including interface tracking, phase change, and droplet dynamics.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from phasta import FlowSolver, FlowConfig, Mesh, MultiphaseConfig

# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')

## Problem Setup

We'll simulate a rising bubble with the following parameters:
- Domain size: 2m × 4m
- Bubble diameter: 0.2m
- Initial position: (1m, 0.5m)
- Liquid density: 1000 kg/m³
- Gas density: 1.2 kg/m³
- Surface tension: 0.072 N/m

In [ ]:
# Create base configuration
config = FlowConfig()
config.domain = {
    'width': 2.0,
    'height': 4.0,
    'mesh_size': 0.02
}

config.flow = {
    'gravity': [0.0, -9.81, 0.0],
    'time_step': 0.001,
    'max_time': 2.0
}

config.boundary_conditions = {
    'left_wall': 'no-slip',
    'right_wall': 'no-slip',
    'top_wall': 'outflow',
    'bottom_wall': 'no-slip'
}

## Rising Bubble Simulation

Let's simulate a rising bubble using the Volume of Fluid (VOF) method.

In [ ]:
# Create mesh
mesh = Mesh.generate_rectangle_2d(
    width=config.domain['width'],
    height=config.domain['height'],
    nx=100,
    ny=200
)

# Plot mesh
plt.figure(figsize=(8, 16))
mesh.plot()
plt.title('Domain Mesh')
plt.show()

In [ ]:
# Configure multiphase flow
multiphase_config = MultiphaseConfig()
multiphase_config.method = 'vof'
multiphase_config.phases = {
    'liquid': {
        'density': 1000.0,
        'viscosity': 0.001,
        'color': 'blue'
    },
    'gas': {
        'density': 1.2,
        'viscosity': 1.8e-5,
        'color': 'red'
    }
}
multiphase_config.interface = {
    'surface_tension': 0.072,
    'curvature_model': 'height_function',
    'advection_scheme': 'geometric'
}
multiphase_config.initial_conditions = {
    'bubble': {
        'type': 'circle',
        'center': [1.0, 0.5],
        'radius': 0.1,
        'phase': 'gas'
    }
}

# Create and run solver
solver = FlowSolver(config, mesh, multiphase_config=multiphase_config)
results = solver.solve()

print("\nRising Bubble Results:")
print(f"Simulation completed: {results.completed}")
print(f"Final time: {results.final_time}")
print(f"Number of time steps: {results.time_steps}")

## Visualize Results

Plot phase distribution, velocity field, and interface curvature.

In [ ]:
# Plot results at different times
times = [0.0, 0.5, 1.0, 1.5, 2.0]
plt.figure(figsize=(15, 12))

for i, t in enumerate(times):
    plt.subplot(3, 2, i+1)
    results.plot_phase_distribution(t)
    plt.title(f'Phase Distribution at t = {t}s')
    plt.colorbar(label='Volume Fraction')

plt.tight_layout()
plt.show()

In [ ]:
# Plot velocity field and interface curvature
plt.figure(figsize=(15, 5))

# Plot velocity field
plt.subplot(121)
results.plot_velocity_magnitude(1.0)
plt.title('Velocity Field at t = 1.0s')
plt.colorbar(label='Velocity (m/s)')

# Plot interface curvature
plt.subplot(122)
results.plot_interface_curvature(1.0)
plt.title('Interface Curvature at t = 1.0s')
plt.colorbar(label='Curvature (1/m)')

plt.tight_layout()
plt.show()

## Droplet Impact Simulation

Now let's simulate a droplet impacting a solid surface.

In [ ]:
# Create mesh for droplet impact
mesh = Mesh.generate_rectangle_2d(
    width=0.1,
    height=0.2,
    nx=50,
    ny=100
)

# Plot mesh
plt.figure(figsize=(5, 10))
mesh.plot()
plt.title('Droplet Impact Mesh')
plt.show()

In [ ]:
# Configure droplet impact
multiphase_config = MultiphaseConfig()
multiphase_config.method = 'vof'
multiphase_config.phases = {
    'water': {
        'density': 1000.0,
        'viscosity': 0.001,
        'color': 'blue'
    },
    'air': {
        'density': 1.2,
        'viscosity': 1.8e-5,
        'color': 'red'
    }
}
multiphase_config.interface = {
    'surface_tension': 0.072,
    'contact_angle': 90.0,
    'curvature_model': 'height_function',
    'advection_scheme': 'geometric'
}
multiphase_config.initial_conditions = {
    'droplet': {
        'type': 'circle',
        'center': [0.05, 0.15],
        'radius': 0.01,
        'phase': 'water',
        'velocity': [0.0, -1.0]
    }
}

# Update flow configuration
config.flow['time_step'] = 0.0001
config.flow['max_time'] = 0.1

# Create and run solver
solver = FlowSolver(config, mesh, multiphase_config=multiphase_config)
results = solver.solve()

print("\nDroplet Impact Results:")
print(f"Simulation completed: {results.completed}")
print(f"Final time: {results.final_time}")
print(f"Number of time steps: {results.time_steps}")

## Visualize Droplet Impact Results

Plot phase distribution and pressure field during impact.

In [ ]:
# Plot results at different times
times = [0.0, 0.02, 0.04, 0.06, 0.08, 0.1]
plt.figure(figsize=(15, 10))

for i, t in enumerate(times):
    plt.subplot(2, 3, i+1)
    results.plot_phase_distribution(t)
    plt.title(f'Phase Distribution at t = {t}s')
    plt.colorbar(label='Volume Fraction')

plt.tight_layout()
plt.show()

In [ ]:
# Plot pressure field during impact
plt.figure(figsize=(15, 5))

# Plot pressure field
plt.subplot(121)
results.plot_pressure(0.02)
plt.title('Pressure Field at t = 0.02s')
plt.colorbar(label='Pressure (Pa)')

# Plot velocity field
plt.subplot(122)
results.plot_velocity_magnitude(0.02)
plt.title('Velocity Field at t = 0.02s')
plt.colorbar(label='Velocity (m/s)')

plt.tight_layout()
plt.show()

## Exercises

1. Try different bubble sizes and densities
2. Experiment with different surface tension values
3. Study the effect of contact angle on droplet impact
4. Compare different interface tracking methods

## Next Steps

- Try the chemical reactions example
- Explore phase change phenomena
- Learn about particle-laden flows